<a href="https://colab.research.google.com/github/SamurAIGPT/langchain-course/blob/main/models/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Models

In the previous lesson we have covered the basics of Langchain. Here is the link to the first lesson https://github.com/SamurAIGPT/langchain-course/blob/main/getting-started/Introduction.ipynb

In this lesson we will talk about different kinds of models including LLMs, Chat Models and Embedding Models

### What is a model ?

A model is a program which is trained to complete a specific task. Since our course is about language tasks the models we will be using are language models. A model is trained on a huge corpus of data

We are not going to cover the process of training as we are going to use already trained models. These pre-trained models are trained on large amounts of data and require a lot of compute to run and thus are called Large Language Models (LLM)

# 1. LLM (Large Language Model)

Now let's talk about LLM. LLM are trained to do language tasks like text generation. There are various LLM in the market but we are going to cover only OpenAI LLM since they are the most popular. OpenAI has 4 such models Davinci, Curie, Ada and Babbage

##### Here is how you can use OpenAI LLM in langchain

##### Let's install necessary libraries and acquire the OPENAI API KEY from the .env file.
---



In [ ]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install python-dotenv

In [1]:
from dotenv import load_dotenv 
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.llms import OpenAI
import os

load_dotenv()

llm = OpenAI(temperature=1, openai_api_key = openai_api_key)
print(llm("Tell me a joke"))



Q: Why don't scientists trust atoms?
A: Because they make up everything!


### Estimating number of tokens

OpenAI models have a context length limiting the size of input data which can be sent to the model. Thus we need to make sure the input text is below that limit before sending to the model. We can do that calculation using the code below

In [3]:
llm.get_num_tokens("what a joke")

3

### Streaming

Streaming is a major concept in LLM which allows you to display output on the go instead of waiting for the full output. Even in the ChatGPT interface you will see content streamed instead of waiting till entire output is generated

Here is a code example for the same. We handle streaming in langchain using a callback handler

In [4]:
from langchain.llms import OpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = OpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0, openai_api_key=openai_api_key)
resp = llm("Write me a poem on beauty.")



Beauty is a thing of wonder,
A sight that can't be denied.
It's a feeling that can't be measured,
A feeling that can't be denied.

It's a thing of beauty that can't be seen,
But can be felt in the heart and soul.
It's a thing of beauty that can't be touched,
But can be seen in the eyes of the beholder.

Beauty is a thing of joy,
A thing that can't be taken away.
It's a thing of beauty that can't be bought,
But can be found in the simplest of things.

Beauty is a thing of love,
A thing that can't be denied.
It's a thing of beauty that can't be measured,
But can be seen in the love of others.

Beauty is a thing of life,
A thing that can't be taken away.
It's a thing of beauty that can't be bought,
But can be found in the simplest of things.

# 2. Chat Models

The second set of models we are going to cover are the chat models. The famous ChatGPT model GPT-3.5 comes under this. The main difference between the previous LLM models and the Chat Models are

* Chat Models are 10x cheaper for api calls
* You can hold a conversation with a chat model like you can with a human which is not possible with the previous LLMs

Since Chat Models can hold a conversation they take a list of chat messages as input instead of plain text like a LLM

Now let's discuss how we can use these Chat Models. Let's do the necessary imports

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

Instead of using a OpenAI class we will be using a ChatOpenAI class to create our chat LLM

In [6]:
chat = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

Input is a bunch of messages. Messages are classified into 3 types

* System Message - This is an initial prompt sent to the model to control the behavior of the model

* Human Message  - Input message of the user

* AI Message     - Message response given by ChatGPT

ChatGPT needs a list of all these messages in the conversation to be able to understand the content and converse further

Now let's see an example where we define the system message and the message input of the user and pass to the chat model. The output generated will be an AI message

We are using a System Prompt to let the model do the task of paraphrasing. This technique of providing the model a prompt to make it perform a task is called Prompt Engineering and can be part of another lesson

In [7]:
messages = [
    SystemMessage(content="You are a helpful assistant that paraphrases the sentence."),
    HumanMessage(content="I love programming.")
]
chat(messages)

AIMessage(content='I have a strong passion for coding.', additional_kwargs={}, example=False)

# Templates in Chat Models

We have discussed templates in lesson 1 which helps us to create dynamic inputs. We can do the same with Chat Models as well. Let's discuss the code for that

We define a system message with input variable task. This task can be dynamically change to do various tasks. For this example we will follow the task of paraphrasing

In [8]:
template="You are a helpful assistant that {task}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat(chat_prompt.format_prompt(task="paraphrases the sentence", text="I love programming.").to_messages())

AIMessage(content='I have a strong passion for coding.', additional_kwargs={}, example=False)

Just like how we used the LLMChain or SequentialChain for LLMs in lesson 1, we can do the same for Chat Models. Thus all the benefits we talked like chaining multiple tasks for LLMs can be achieved with Chat Models as well Here is an example

In [9]:
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(task="paraphrases the sentence", text="I love programming.")

'I have a strong passion for coding.'

### Streaming with Chat Models

We disucussed about how streaming is useful in the above section of LLMs. Now let's see how we can do the same with Chat Models

In [11]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
chat = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0, openai_api_key=openai_api_key)
resp = chat([HumanMessage(content="Write me a poem on beauty.")])

In the realm of beauty, let us embark,
Where nature's wonders ignite a spark.
A tapestry woven with colors so grand,
A symphony of grace, across the land.

Behold the sunrise, a celestial dance,
As golden rays upon the earth enhance.
The sky ablaze with hues of crimson fire,
A masterpiece that fills our hearts' desire.

The moon, a pearl, adorns the velvet night,
Casting its glow, a beacon of pure light.
Stars twinkle like diamonds, a celestial choir,
Guiding our dreams, igniting our desire.

In every flower, a secret is concealed,
Petals unfurl, their fragrant love revealed.
A delicate ballet, as they sway and bloom,
Their vibrant beauty, a sweet, divine perfume.

The mountains rise, majestic and serene,
Their peaks touch the heavens, a sight unseen.
Cloaked in mist, they stand with silent grace,
A testament to nature's perfect embrace.

The ocean's vastness, a mesmerizing sight,
Its waves crashing, a symphony of might.
Endless depths, where mysteries reside,
A captivating beauty, imp

# Embedding models

So far we have talked about text generation models but now we are going to talk about a completely different kind of model called Embedding models

### Embeddings

First we need to understand what is an embedding. An embedding is generally associated with a piece of text and it represents the properties of text

Just to give an example, let's consider the words good, best, bad. If we find the embeddings of these words we observe that embeddings of good and best are close while embedding of bad is far. The reason being embedding of a word has knowledge of the meaning of the word. Thus words with similar meanings have similar embeddings

Embeddings also have an interesting as can be seen below. Let's consider E(x) as Embedding of word x

E(king) - E(male) + E(female) ~= E(queen)

What this represents is if we subtract the embedding of word male from word king and add the embedding of word female it will be quite close to embedding of word queen. As humans we can understand this intuitively as removing male gender and adding female gender to king makes it a queen but now machines have the capability to understand such complex relations

### Use-cases

Now that we have an idea of what is embeddings, the task of a embeddings model is to create these embeddings for the text input provided. A model which generates embedding which can show properties like the ones we discussed above and more is considered a good model.

Once these embeddings are generated, we can use it to perform tasks like semantic search similar to how apps like Chatbase, PDF.ai, SiteGPT work. You can creating embeddings for all your documents or webpages and when user asks a query you can fetch the relevant pieces and send to the user

Now let's discuss it with the help of an example

In [12]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
text = "This is dummy content."
doc_result = embeddings.embed_documents([text])

As you can see, the output is a vector which is a respresentation of text "This is dummy content". To identify if two sentences are similar, we can calculate the distance between these vectors. If the distance is small, then the words are of similar meanings

In [13]:
doc_result

[[0.009624725285748391,
  0.005727740051113911,
  -0.025073372030961615,
  -0.009651258666114387,
  -0.01194633366916001,
  0.014234776724098007,
  -0.03536804794148363,
  -0.02598874850787875,
  -0.01833407409457965,
  -0.03796824843683496,
  0.0040395986616420234,
  0.0073362826964716845,
  -0.015587942801183084,
  0.009505328799391739,
  -0.007289850677815065,
  -0.00630482524159042,
  0.0176442248315154,
  -0.021876184447214113,
  0.010752364148980578,
  -0.01796261608268153,
  -0.0033945231973768458,
  0.005207036198439332,
  -0.009737489823997417,
  0.01796261608268153,
  -0.013272967297201673,
  -0.007953167026174605,
  0.019673973481481727,
  -0.027222518098607176,
  0.014168445135828182,
  -0.004908543585563829,
  0.03130855064155098,
  -0.020722014997583676,
  -0.00595326773062809,
  -0.03709267109914424,
  0.011468745861088238,
  0.0009352765703992915,
  0.004540402944703397,
  -0.0366681481891593,
  0.01699417470767757,
  0.001897085822672642,
  -0.02090774307221015,
  0.00